In [142]:
import numpy as np
from abc import ABC, abstractmethod

In [143]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [257]:
import distributions as dist
import interference as fere

In [258]:
# get RNG
rng  = np.random.default_rng()

In [261]:
sfdist = dist.Uniform(0, 10)
print(sfdist)
sfdist()

UniformDistribution({'low': 0, 'high': 10, 'seed': None})


4.987802887478647

DiscreteDistribution({'a': 4, 'p': None, 'replace': True, 'seed': None})

# Time windowns and arrival processes

Define a `TimeWindow` over which to sample, and choose an `ArrivalProcess` process over it.

In [146]:
# make a time window
window = dist.TimeWindow(0, 5, buffer=2)

# specify arrival process
arrivals = dist.PoissonArrivals(window, rate=1.3)

print(arrivals)

HomogeneousPoissonArrivals(TimeWindow(0, 5, buffer=2), rate=1.3)


In [147]:
# sample arrival times
arrs = arrivals.sample(seed=rng)

print(arrs)

[array([ 2.77731509,  6.54580771,  6.42488532,  3.45351156, -0.57500979,
        2.45297018, -1.97166117,  1.32420363, -1.42583077])]


# A parameter colleciton

In [148]:
# parameters for a LoRaWAN
wan = fere.LoRaParameters()

print(wan)

A LoRa Parameters object with params:
	 nChannels: 1
	 freq: 915
	 bw: 125
	 sf: [7, 8, 9, 10]
	 overhead: 13
	 maxPow: 30
	 dwellTime: 400
	 dutyCycle: None


# Example: sampling Traffic

In [149]:
# equip arrival times with Traffic parameters
nPackets = len(start[0])
airtime = rng.uniform(0.1, 0.2, size=nPackets)
channel = rng.choice(8, size=nPackets)
sf = 7 + rng.choice(4, size=nPackets)
power = rng.normal(loc=-90, scale=10, size=nPackets)

# make traffic object
traffic = fere.Traffic(nPackets, start[0], airtime,
                      channel, sf, power)

print(traffic)

Traffic(nObs=11)


## Network - A Traffic Generating Process

We keep the `LoRaWAN` class.

Distributionally-equipped traffic-generating networks are children!
- Distribution over channels.
- Distribution over SFs.
- Distribution over payloads(s)

In [152]:
# make LoRaParameters instance
myLoRaWAN = fere.LoRaParameters(nChannels=4)
print(myLoRaWAN)

A LoRa Parameters object with params:
	 nChannels: 4
	 freq: 915
	 bw: 125
	 sf: [7, 8, 9, 10]
	 overhead: 13
	 maxPow: 30
	 dwellTime: 400
	 dutyCycle: None


# Traffic Generators

In [138]:
class TrafficGenerator(ABC):
    """
    Class of traffic-generating objects.
    
    ``TrafficGenerators`` are compositions of objects. A typical generator
    takes an ``ArrivalProcess`` and a ``ParameterDistribution``.
    ``Traffic``-objects are generated by sampling arrivals and equipping
    these with sampled parameters. Dependence between arrivals and
    parameters should also be supported.
    """
    
    @abstractmethod
    def __call__(self, seed=None):
        """
        Get a sample.
        """
        
class IndependentTraffic(TrafficGenerator):
    """A class for LoRa traffic generation.
    independent sampling of wireless parameters.
    
    Attributes
    ----------
    
    arrivals : ArrivalProcess
        An arrival process from ``loraplan.distributions``.
    parameters : ParameterDistribution
        A distribution over wireless parameters.
    """
    def __init__(self, arrivals=None, paramDist=None, loraParams=None):
        """
        Instantiate an independent LoRa traffic generator object.

        Parameters
        ----------
        arrivals : ArrivalProcess
            An arrival process describing for sampling nPackets and arrival times.

        paramDist : parameterDistribution
            A distribution from which to sample LoRa
            
        loraParams: LoRaWANParameters, optional
            A parameter object against which to verify validity of distributions.
        """
        
        self.arrivals = arrivalProcess
        self.parameters = paramDist
        
    def __call__(self, seed=None):
        pass

HomogeneousPoissonArrivals(TimeWindow(0, 5, buffer=2), rate=1.3)

## Parameter Distribution

In [140]:
class Distribution(ABC):
    """
    Class for custom made probability distributions.
    
    Allows attaching parameters and a seed to the distribution instance
    so as to streamline object compositions using ``Distribution``-objects.
    
    Notes
    -----
    The `__call__` function is used for sampling.
    The parameters and defaults should be explicit.
    Each distribution uses a `numpy.random.generator`,
    and should accept and optional `seed` to which
    a `seed` string or a `numpy.random.generator` can be
    passed.
    
    """
    
    @classmethod
    def __call__(self):
        pass

In [170]:
class Normal(Distribution):
    """A normal distribution.
    """
    
    def __init__(self, loc=0, scale=1, seed=None):
        """Instantiate normal distribution.
        """
        self.loc = loc
        self.scale = scale
        self.seed = seed
        
    def __repr__(self):
        return "Normal(%s)" % str(self.__dict__) 
        
        
    def __call__(self, size=None, *, loc=None, scale=None, seed=None):
        """
        Sample of independent univariate normal random variables.
        
        Parameters
        ----------
        
        loc : float or array_like
            Location parameter(s)
        scale : float or array_like
            Scale parameter(s)
        size : int or tuple of ints
            Size of sample
        seed : int, optional
            A seed to override the Distribution's internal seed.
        
        Returns
        -------
        A normal sample of given size.
            
        See Also
        --------
        numpy.random.normal
        """ 
        if loc is None:
            loc = self.loc
        if scale is None:
            scale = self.scale
        
        # override self.seed if provided
        if not seed:
            seed = self.seed
        rng = np.random.default_rng(seed)
        
        
        return rng.normal(loc=loc, scale=scale, size=size)

In [169]:
# demo of Normals
norm1 = Normal(seed=rng)
norm2 = Normal(loc=10, scale=5)
print(norm1)
print(norm2)

Normal({'loc': 0, 'scale': 1, 'seed': Generator(PCG64) at 0x15AD8CF20})
Normal({'loc': 10, 'scale': 5, 'seed': None})


array([-0.54594771,  0.51718241,  0.05694123, -0.75834263, -1.06928845,
       -0.1356848 , -1.42669576,  0.46909474,  0.01720726, -2.02090593])

In [ ]:
class WirelessDistribution(ABC):
    """
    A distribution over wireless parameters.
    
    Attributes
    ----------
    
    Notes
    -----
    
    This class models any distribution over wireless
    parameters, conditional on arrivals. I.e. the
    number of packets and their arrival times.
    """
    
    @classmethod
    def sample(self):
        pass    

In [ ]:
class Independent(WirelessDistribution):
    """
    A distribution over LoRa Tx parameters in which
    each type of parameter are mutuall independent.
    
    Attributes
    ----------
    payloadDist
        A distribution over payloads.
    channelDist
        A distribution over channels.
    spreadingDidst
        A distribution over spreading factors.
    powerDist
        A distribution over power.
    """
    
    
    

In [ ]:
class LoRaProcess():
    """
    A traffic-generating process.
    
    Attributes
    ----------
    network : LoRaWAN
        A LoRaWAN specifying wireless parameters of traffic.
    arrivals : ArrivalProcess
        Arrival process of wireless traffic.
    paramDist
        A distribution over network parameters.
        
    Notes
    -----
    This is a data-generating process. The main purpose of the class
    is to collect all relevant components in one place.
    
    A general `paramDist` is prefered over a distribution over individual
    wireless parameters, as the more general distribution permits
    arbitrary joint distrtibutions over groups of parameters. E.g. when
    power and SF are correlated.
    """
    
    def __init__(self):
        """
        Instantiate LoRaProcess.
        """
        

In [ ]:
def 